In [10]:
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from descriptors import preprocess_image, extract_descriptor, extract_descriptors

# Paths
IMG_FOLDER = "../Data/Week3/qst1_w3/"
IMG_FOLDER_GT = "../Data/Week3/BBDD/"
DESC_GT_PATH = "results/descriptors_gt.pkl"
RESULTS_PATH = "results/query_to_gt_top10.pkl"

K = 10  # number of best matches to retrieve

# --- 1. Load or compute GT descriptors ---
if os.path.exists(DESC_GT_PATH):
    print(f"✅ Loading GT descriptors from {DESC_GT_PATH}")
    with open(DESC_GT_PATH, "rb") as f:
        data = pickle.load(f)
        desc_gt = data["desc_gt"]
        gt_names = data["gt_names"]
else:
    print("🧠 Computing GT descriptors...")
    desc_gt, gt_names = extract_descriptors(IMG_FOLDER_GT, preprocess=False)
    
    # Save to file for later use
    os.makedirs(os.path.dirname(DESC_GT_PATH), exist_ok=True)
    with open(DESC_GT_PATH, "wb") as f:
        pickle.dump({"desc_gt": desc_gt, "gt_names": gt_names}, f)
    print(f"💾 Saved GT descriptors to {DESC_GT_PATH}")

# --- 2. Process query images ---
image_names = sorted([f for f in os.listdir(IMG_FOLDER) if f.endswith('.jpg')])
results = []

for img_name in image_names:
    print(f"\nProcessing {img_name} ...")
    img_path = os.path.join(IMG_FOLDER, img_name)
    img = cv2.imread(img_path)

    if img is None:
        print(f"⚠️ Skipping {img_name}: could not read image.")
        continue

    # Preprocess and extract descriptor
    img = preprocess_image(img)
    desc = extract_descriptor(img)

    # --- 3. Compare with GT descriptors using cosine similarity ---
    similarities = cosine_similarity([desc], desc_gt)[0]
    
    # Get indices of top-K most similar GT images
    top_k_indices = np.argsort(-similarities)[:K]  # sort descending
    results.append(top_k_indices.tolist())

# --- 4. Save results ---
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
with open(RESULTS_PATH, "wb") as f:
    pickle.dump(results, f)

print(f"\n✅ Saved top-{K} retrieval results to {RESULTS_PATH}")


✅ Loading GT descriptors from results/descriptors_gt.pkl

Processing 00000.jpg ...

Processing 00001.jpg ...

Processing 00002.jpg ...

Processing 00003.jpg ...

Processing 00004.jpg ...

Processing 00005.jpg ...

Processing 00006.jpg ...

Processing 00007.jpg ...

Processing 00008.jpg ...

Processing 00009.jpg ...

Processing 00010.jpg ...

Processing 00011.jpg ...

Processing 00012.jpg ...

Processing 00013.jpg ...

Processing 00014.jpg ...

Processing 00015.jpg ...

Processing 00016.jpg ...

Processing 00017.jpg ...

Processing 00018.jpg ...

Processing 00019.jpg ...

Processing 00020.jpg ...

Processing 00021.jpg ...

Processing 00022.jpg ...

Processing 00023.jpg ...

Processing 00024.jpg ...

Processing 00025.jpg ...

Processing 00026.jpg ...

Processing 00027.jpg ...


KeyboardInterrupt: 

In [7]:
print(results)

[[90, 221, 175, 6, 184, 263, 235, 48, 204, 201], [131, 64, 86, 71, 107, 30, 98, 238, 265, 247], [212, 182, 116, 268, 48, 24, 120, 1, 209, 194], [28, 59, 185, 227, 152, 134, 180, 81, 10, 83], [136, 274, 78, 106, 254, 165, 30, 46, 234, 250], [86, 172, 238, 93, 61, 107, 14, 125, 89, 81], [113, 258, 56, 130, 116, 184, 268, 183, 239, 176], [263, 235, 210, 201, 1, 221, 48, 184, 22, 243], [36, 40, 49, 209, 120, 236, 47, 87, 24, 44], [101, 60, 174, 102, 252, 232, 261, 52, 128, 77], [65, 203, 98, 107, 247, 14, 30, 251, 265, 200], [62, 179, 219, 231, 9, 146, 50, 137, 27, 55], [35, 36, 236, 65, 250, 164, 98, 253, 177, 44], [47, 36, 232, 236, 280, 164, 49, 147, 108, 40], [183, 268, 202, 209, 116, 194, 56, 178, 48, 1], [193, 97, 4, 182, 22, 217, 177, 44, 253, 164], [184, 210, 221, 269, 235, 263, 243, 241, 255, 141], [87, 265, 120, 44, 209, 236, 24, 65, 40, 54], [219, 179, 146, 247, 148, 281, 30, 216, 155, 14], [124, 185, 82, 29, 54, 227, 172, 81, 64, 59], [120, 40, 36, 65, 265, 11, 44, 209, 24, 251